# Sistemas RAG

Los sistemas RAG permiten mejorar la eficacia de los sistemas de preguntas respuestas (query answering). Por un lado reducen las alucinaciones que los modelos puedan tener y, por otro lado, permiten dar una evidencia empírica de la proveniencia de la información sobre la que se basa la respuesta. Para ello, los sistemas RAG necesitan una base documental sobre la que articular la respuesta. Cuando hablamos de documentos, no nos referimos a ficheros en sí, sino a conjuntos de información como pueden ser párrafos, frases, páginas de texto, etc. dependiendo de la granularidad que se le quiera dar a dicha unidad. El flujo del modelo RAG, dada una consulta y una base documental, sería:

* Retrieval: buscar fragmentos relevantes en la base documental que tengan relación con la pregunta.
* Augmentation: pasar los fragmentos recuperados como contexto adicional al modelo generador.
* Generation: el modelo genera la respuesta teniendo en cuenta el contexto documental

A continuación vamos a ver varias opciones para la parte de Retrieval (Recuperación), como comparar dichos sistemas, y la mejora que incorporan al módulo de generación.

Para ello, vamos a reutilizar parte del código visto en el notebook de [Búsqueda Dispersa y Densa](https://github.com/cbadenes/curso-pln/blob/main/notebooks/08_Busqueda_Dispersa_y_Densa.ipynb) y en el de [RAG Avanzado](https://github.com/cbadenes/curso-pln/blob/main/notebooks/08_RAG_Avanzado.ipynb).

In [ ]:
from huggingface_hub import login
token = ""
print("Hugging Face logging")
login(token)

In [ ]:
import torch
import os
device_setup= "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
print("Using: ",device_setup)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# RAG: Retrieval-Augmented Generation

In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from typing import List, Tuple
import nltk

# Descarga de recursos necesarios
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)

class TextPreprocessor:

    @classmethod
    def preprocess(cls, text: str, lang='spanish') -> str:
        text = text.lower()
        # Tokenizar usando NLTK
        tokens = word_tokenize(text)
        # Eliminar stopwords usando NLTK
        stop_words = set(stopwords.words(lang))
        tokens = [t for t in tokens if t not in stop_words]

        return ' '.join(tokens)


Como vamos a trabajar con distintos tipos de retrievers, para que todos se puedan utilizar de la misma manera (métodos con el mismo nombre), vamos a crear una estructura de herencia dónde todos las clases de los retriever tendrán que implementar los métodos de la siguiente clase abstracta/interfaz.

In [ ]:
from abc import ABC, abstractmethod

class Retriever(ABC):

    def __init__(self, name='abstract_retriever'):
        self.name = name

    def get_name(self):
        return self.name

    """
    Este método recibe un conjunto de documentos y los indexa para poder realizar búsquedas posteriores
    """
    @abstractmethod
    def build_index(self, documents: List[str], lang: str = 'spanish'):
        pass

    """
        Este método búsca los documentos relevantes para una query.
        Devuelve una lista con el la posición (index) del documento encontrado y su score de relevancia.
    """
    @abstractmethod
    def search(self, query: str, top_k: int = 3, lang:str = 'spanish') -> List[Tuple[int, float]]:
        pass

    """
        Este método búsca los documentos relevantes para una query.
        Devuelve los documentos que considera relevantes.
    """
    @abstractmethod
    def search_documents(self, query: str, top_k: int = 3, lang:str = 'spanish') -> List[str]:
        pass

## SparseRetriever

A continuación podemos ver un ejemplo de sparse retriever basado en TF-IDF para vectorizar los textos y el uso del modelo NearestNeighbors con la distancia coseno para recuperar los documentos relevantes. Es importante también observar como se ha indicado la herencia de la clase Retriever.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


'''
    * Búsqueda eficiente: El uso de NearestNeighbors con una métrica de similitud como el coseno permite realizar búsquedas rápidas.
    * TF-IDF como base: Las palabras más relevantes en cada documento obtienen un peso mayor, mejorando la precisión de la búsqueda.
'''
class SparseRetrieverNM(Retriever):

    def __init__(self):
        super().__init__("sparse_retriever_nm")
        self.vectorizer = TfidfVectorizer()
        self.nn_model = NearestNeighbors(n_neighbors=5, metric="cosine", algorithm="auto")

    """
    Construye el índice usando TF-IDF
    """
    def build_index(self, documents: List[str], lang: str = 'spanish'):
        self.documents = documents
        # Limpiar tokens innecesarios
        processed_docs = [TextPreprocessor.preprocess(doc, lang) for doc in self.documents]
        # Generar embeddings dispersos TF-IDF
        self.tfidf_matrix = self.vectorizer.fit_transform(processed_docs)
        # Construir un modelo de búsqueda eficiente
        self.nn_model.fit(self.tfidf_matrix)

    def search(self, query: str, top_k: int = 5, lang: str = 'spanish') -> List[Tuple[int, float]]:
        # Vectorizar la consulta
        processed_query = TextPreprocessor.preprocess(query, lang)
        query_vector = self.vectorizer.transform([processed_query])

        # Encontrar los vecinos más cercanos
        distances, indices = self.nn_model.kneighbors(query_vector, n_neighbors=top_k)
        # Retornar resultados como documentos y distancias inversas (para similitud)
        return [(idx, score) for idx, score in zip(indices[0], distances[0])][::-1]

    def search_documents(self, query: str, top_k: int = 3, lang: str = 'spanish') -> List[str]:
        relevant_documents = self.search(query, top_k, lang)
        return [self.documents[idx] for idx, score in relevant_documents]

#### Tarea RAG1

 Utilizando de base el código de las clases pasadas, implementar un SparseRetriever basado en el vectorizador BM25. Es importante que la clase herede de `Retriever`


In [ ]:
## TODO: desarrollar un SparseRetriever
class SparseRetriever(Retriever):
    """
    Implementa búsqueda dispersa usando BM25.
    """
    def __init__(self):
        super().__init__('sparse_retriever')


    def build_index(self, documents: List[str], lang:str = 'spanish'):
         pass


    def search(self, query: str, top_k: int = 3, lang:str = 'spanish') -> List[Tuple[int, float]]:
        pass

    def search_documents(self, query: str, top_k: int = 3, lang:str = 'spanish') -> List[str]:
        pass


## Dense Retriever:


#### Tarea RAG2

De manera similar a antes, implementar un DenseRetriever que herede de la clase Retriever. La clase tiene que ser modular para poder, internamente, utilizar distintos modelos. En particular, querremos instanciar un DenseRetriever con el modelo 'sentence-transformers/all-MiniLM-L6-v2' (384 dimensional dense vector) y otro con el modelo 'distiluse-base-multilingual-cased-v1' (512 dimensional dense vector)

In [ ]:
# TODO: implementar el DenseRetriever
class DenseRetriever(Retriever):

    def __init__(self):
        super().__init__('dense_retriever')
        pass


    def build_index(self, documents: List[str], lang: str = 'spanish'):
        pass


    def search(self, query: str, top_k: int = 3, lang: str = 'spanish') -> List[Tuple[int, float]]:
        pass


    def search_documents(self, query: str, top_k: int = 3, lang: str = 'spanish') -> List[str]:
       pass


### Combinación de Técnicas Dispersas y Densas


#### Tarea RAG3

De manera similar a antes, implementar un HybridRetriever que herede de la clase Retriever. La clase tiene que ser combinar un *dense retriever* y un *sparse retriever*. En particular, querremos instanciar un DenseRetriever con el modelo 'sentence-transformers/all-MiniLM-L6-v2' (384 dimensional dense vector) y otro con el modelo 'distiluse-base-multilingual-cased-v1' (512 dimensional dense vector)

In [ ]:
class HybridRetriever(Retriever):

    def __init__(self):
        super().__init__('hybrid_retriever')
        pass

    def build_index(self, documents: List[str], lang: str = 'spanish'):
        pass

    def search(self, query: str, top_k: int = 3, lang: str = 'spanish') -> List[Tuple[int, float]]:
        pass

    def search_documents(self, query: str, top_k: int = 3, lang: str = 'spanish') -> List[str]:
     pass

## Evaluación de sistemas RAG

Para evaluar estos sistemas, necesitamos un dataset que posea una pregunta, un conjunto de textos, y una relación entre la pregunta y cuáles de dichos textos son 'relevantes' para responder la pregunta. Para ello, podemos usar los datasets de `rungalileo/ragbench`, por ejemplo el `techqa`. Vamos a escribir el código para que por un lado se extraigan todos los posibles documentos del dataset y, por otro lado, se emparejen las consultas con los fragmentos relevantes.

In [ ]:
from datasets import load_dataset, Dataset
# load train/validation/test splits of individual subset
ragbench = load_dataset("rungalileo/ragbench", "techqa", split=["test"])

# Preparar documentos, consultas y relevancias manualmente
def format_dataset(dataset: List[Dataset]):
      # Aplanar los documentos si son listas de listas
    documents = []
    for doc in dataset["documents"]:
        if isinstance(doc, list):
            documents.extend(doc)  # Añadir documentos individuales
        else:
            documents.append(doc)
    queries = {dataset["id"][idx] : question for idx, question in enumerate(dataset['question'])}
    relevant_docs = {dataset["id"][idx] : response for idx, response in enumerate(dataset["documents"])}
    return {'documents': documents, 'queries': queries, 'gold_std': relevant_docs}

dataset = format_dataset(ragbench[0])
for idx_doc, value in dataset['gold_std'].items():
    print("document_id: ", idx_doc)
    print("documents: ", len(value))
    print("related query: ", dataset['queries'][idx_doc])
    break


Por último vamos a escribir un método que compruebe la eficacia de un conjunto de retrievers. Nótese que, al contrario que pasaba con los query answering, para los Retrievers es posible calcular métricas no difusas (fuzzy).

In [ ]:
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score
from sentence_transformers import SentenceTransformer, util
import numpy as np
import matplotlib.pyplot as plt

def evaluate_retrievers(retrievers, documents, queries, gold_std, top_k=5):
    results = {}
    for retriever in retrievers:
        retriever_name = retriever.get_name()
        print(f"Evaluando {retriever_name}...")
        retriever.build_index(documents)
        model_results = []

        for query_id, query in queries.items():
            if query_id not in gold_std:
                continue
            ground_truth = gold_std[query_id]
            retrieved_docs = retriever.search_documents(query, top_k=top_k)

            # Calcular métricas de evaluación
            y_true = [1 if doc_found in ground_truth else 0 for doc_found in retrieved_docs]
            y_pred = [1] * len(retrieved_docs)
            precision = precision_score(y_true, y_pred, zero_division=0)
            recall = recall_score(y_true, y_pred, zero_division=0)
            f1 = f1_score(y_true, y_pred, zero_division=0)

            model_results.append({"precision": precision, "recall": recall, "f1": f1})

        avg_precision = np.mean([r["precision"] for r in model_results])
        avg_recall = np.mean([r["recall"] for r in model_results])
        avg_f1 = np.mean([r["f1"] for r in model_results])

        results[retriever_name] = {
            "precision": avg_precision,
            "recall": avg_recall,
            "f1": avg_f1
        }

    return results

def plot_comparison(results, min_y=0.7, max_y=1.0):
    metrics = ["precision", "recall", "f1"]
    models = list(results.keys())

    for metric in metrics:
        values = [results[model][metric] for model in models]
        plt.figure(figsize=(10, 6))
        plt.bar(models, values)
        plt.title(f"Comparación de {metric.capitalize()}", fontsize=14)
        plt.xlabel("Modelos", fontsize=12)
        plt.ylabel(metric.capitalize(), fontsize=12)
        plt.ylim(min_y, max_y)
        plt.xticks(rotation=45, fontsize=10)
        plt.yticks(fontsize=10)
        plt.tight_layout()
        plt.show()

### Pruebas manuales

Vamos a analizar, usando el dataset anterior, los resultados de manera manual que un retriever puede obtener.

In [ ]:
retriever = SparseRetrieverNM()
retriever.build_index(dataset['documents'])

documents_found = retriever.search_documents(dataset['queries']['techqa_DEV_Q243'])
expected_docs = dataset['gold_std']['techqa_DEV_Q243']
print("Total documents to be found: ", len(expected_docs))
for docs_found in documents_found:
    for expected_doc in expected_docs:
        if expected_doc in docs_found:
            print("relevant document found!")


### Pruebas de benchmark

A continuación vamos a evaluar todos los retriever que hemos desarrollado para comprobar cual se comporta mejor en los distintos datasets del repositorio `rungalileo/ragbench`

In [ ]:
# Crear instancias de retrievers
retrievers = [SparseRetriever(), SparseRetrieverNM(), DenseRetriever(), DenseRetriever( model='distiluse-base-multilingual-cased-v1'), HybridRetriever(), HybridRetriever(model='distiluse-base-multilingual-cased-v1')]

In [ ]:
# load train/validation/test splits of individual subset
ragbench= load_dataset("rungalileo/ragbench", "techqa", split=["test"])
dataset = format_dataset(ragbench[0])

# Evaluar los retrievers
results = evaluate_retrievers(retrievers, dataset['documents'], dataset['queries'], dataset['gold_std'], top_k=5)

# Mostrar y graficar resultados
for model, metrics in results.items():
    print(f"{model}: Precisión={metrics['precision']:.4f}, Recall={metrics['recall']:.4f}, F1={metrics['f1']:.4f}")

plot_comparison(results, min_y=0.0, max_y=1.0)

In [ ]:
# load train/validation/test splits of individual subset
ragbench= load_dataset("rungalileo/ragbench", "covidqa", split=["test"])
dataset = format_dataset(ragbench[0])

# Evaluar los retrievers
results = evaluate_retrievers(retrievers, dataset['documents'], dataset['queries'], dataset['gold_std'], top_k=5)

# Mostrar y graficar resultados
for model, metrics in results.items():
    print(f"{model}: Precisión={metrics['precision']:.4f}, Recall={metrics['recall']:.4f}, F1={metrics['f1']:.4f}")

plot_comparison(results, min_y=0.0, max_y=1.0)

In [ ]:
ragbench= load_dataset("rungalileo/ragbench", "finqa", split=["test"])
dataset = format_dataset(ragbench[0])

# Evaluar los retrievers
results = evaluate_retrievers(retrievers, dataset['documents'], dataset['queries'], dataset['gold_std'], top_k=5)

# Mostrar y graficar resultados
for model, metrics in results.items():
    print(f"{model}: Precisión={metrics['precision']:.4f}, Recall={metrics['recall']:.4f}, F1={metrics['f1']:.4f}")

plot_comparison(results, min_y=0.0, max_y=1.0)

In [ ]:
# load train/validation/test splits of individual subset
ragbench= load_dataset("rungalileo/ragbench", "hotpotqa", split=["test"])
dataset = format_dataset(ragbench[0])

# Evaluar los retrievers
results = evaluate_retrievers(retrievers, dataset['documents'], dataset['queries'], dataset['gold_std'], top_k=5)

# Mostrar y graficar resultados
for model, metrics in results.items():
    print(f"{model}: Precisión={metrics['precision']:.4f}, Recall={metrics['recall']:.4f}, F1={metrics['f1']:.4f}")

plot_comparison(results, min_y=0.0, max_y=1.0)